In [1]:
from src.stoch_modelling import JDMSimulator, CorrelatedJDMSimulator, CIRJumpModel
from src.linear_factor_model import LinearFactorModel
import pandas as pd
import numpy as np
from src.data_handling import load_csv_data, filter_data_by_year_range, append_string_to_elements

In [2]:
DATA_PATH = "DATA/processed/df.csv"

df = load_csv_data(DATA_PATH)
df.head()

MODELING_YEARS = (2015, 2025)

df = filter_data_by_year_range(df, *MODELING_YEARS)
df.head()

,AAPL_Open,AAPL_High,AAPL_Low,AAPL_Close,AAPL_Adj Close,AAPL_Volume,AMZN_Open,AMZN_High,AMZN_Low,AMZN_Close,...,VIX_High,VIX_Low,VIX_Close,VIX_Adj Close,IRX_Open,IRX_High,IRX_Low,IRX_Close,IRX_Adj Close,Year
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,27.847500,27.860001,26.837500,27.332500,24.531765,212818400.0,15.6290,15.7375,15.3480,15.4260,...,20.139999,17.049999,17.790001,17.790001,0.030,0.030,0.015,0.015,0.015,2015
2015-01-05,27.072500,27.162500,26.352501,26.562500,23.840672,257142000.0,15.3505,15.4190,15.0425,15.1095,...,21.290001,19.190001,19.920000,19.920000,0.008,0.018,0.003,0.003,0.003,2015
2015-01-06,26.635000,26.857500,26.157499,26.565001,23.842909,263188400.0,15.1120,15.1500,14.6190,14.7645,...,22.900000,19.520000,21.120001,21.120001,0.023,0.023,0.018,0.020,0.020,2015
2015-01-07,26.799999,27.049999,26.674999,26.937500,24.177242,160423600.0,14.8750,15.0640,14.7665,14.9210,...,20.719999,19.040001,19.309999,19.309999,0.023,0.025,0.020,0.020,0.020,2015
2015-01-08,27.307501,28.037500,27.174999,27.972500,25.106180,237458000.0,15.0160,15.1570,14.8055,15.0230,...,18.090000,16.990000,17.010000,17.010000,0.020,0.020,0.015,0.018,0.018,2015


In [3]:
CORR_JDM_FACTORS = ["AAPL_Close", "MSFT_Close", "AMZN_Close"]
LFM_FACTORS = ["AAPL_Close", "MSFT_Close", "AMZN_Close", "VIX_Close", "IRX_Close"]
N_SIM = 1000
n_days = 7

temp = df.iloc[:50]


class StochasticVolForecast:
    def __init__(self, data, lfm_factors, corr_jdm_factors, jdm_factor = 'VIX_Close', cirj_factor = 'IRX_Close', lfm_target = 'SPX_Close'):


        self.data             = data
        self.lfm_factors      = lfm_factors
        self.corr_jdm_factors = corr_jdm_factors
        self.jdm_factor       = jdm_factor
        self.cirj_factor      = cirj_factor
        self.lfm_target       = lfm_target
        self.initial_price    = data[lfm_target].iloc[-1]

        self.corr_jdm_model = None
        self.jdm_model      = None
        self.cirj_model     = None
        self.lfm_model      = None

    def fit_models(self):
        
        # Initialise models
        corr_jdm_model = CorrelatedJDMSimulator()
        jdm_model      = JDMSimulator(model = "Merton")
        cirj_model     = CIRJumpModel()
        lfm_model      = LinearFactorModel()

        # Estimating parameters for models
        corr_jdm_model.estimate_parameters(self.data[CORR_JDM_FACTORS].values)
        jdm_model.estimate_parameters(self.data[self.jdm_factor].values)
        cirj_model.estimate_parameters(self.data[self.cirj_factor].values)
        lfm_model.fit(self.data[self.lfm_target].values, self.data[self.lfm_factors].values)

        # set initial price
        corr_jdm_model.initial_prices = temp[CORR_JDM_FACTORS].iloc[0]
        jdm_model.initial_price = temp['VIX_Close'].iloc[0]
        cirj_model.initial_rate = temp['IRX_Close'].iloc[0]

        # setting object attributes
        self.corr_jdm_model = corr_jdm_model
        self.jdm_model      = jdm_model
        self.cirj_model     = cirj_model
        self.lfm_model      = lfm_model

    def simulate_portfolio_path(self, n_days):
        corr_jdm_sim = self.corr_jdm_model.simulate(n_days)
        jdm_sim      = self.jdm_model.simulate(n_days-1).reshape(-1,1)
        cirj_sim     = self.cirj_model.simulate(n_days-1).reshape(-1,1)

        sim_concat = np.hstack((corr_jdm_sim, jdm_sim, cirj_sim))

        return self.lfm_model.predict(sim_concat)


    def forecast_volatility(self, n_days, n_simulations = 10_000):
            """Forecast volatility using Monte Carlo simulations."""
            volatilities = []
            for _ in range(n_simulations):
                portfolio_path = self.simulate_portfolio_path(n_days)
                returns = np.diff(np.log(portfolio_path))
                volatility = np.std(returns)
                volatilities.append(volatility)
            return np.mean(volatilities)*self.initial_price

In [4]:
stoch_vol = StochasticVolForecast(temp, LFM_FACTORS, CORR_JDM_FACTORS)
stoch_vol.fit_models()
stoch_vol.forecast_volatility(7, 1000)

c:\Users\bruce\Desktop\Imperial\Sem 3\AP\CODE\src\stoch_modelling.py:195: RuntimeWarning: divide by zero encountered in log
  likelihoods.append(np.log(likelihood))
c:\Users\bruce\anaconda3\lib\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


5.241002745281607

In [5]:
asset_sim = asset_model.simulate(n_days)
vix_sim = vix_model.simulate(n_days-1).reshape(-1,1)
irx_sim = irx_model.simulate(n_days-1).reshape(-1,1)



NameError: name 'asset_model' is not defined

In [ ]:
lfm.predict(sim_concat)

array([2047.12974742, 2043.78750159, 2052.28007512, 2048.02645186,
       2045.85958385, 2038.51582822, 2040.34372982])

Date
2015-01-02    2058.199951
2015-01-05    2020.579956
2015-01-06    2002.609985
2015-01-07    2025.900024
2015-01-08    2062.139893
2015-01-09    2044.810059
2015-01-12    2028.260010
2015-01-13    2023.030029
2015-01-14    2011.270020
2015-01-15    1992.670044
2015-01-16    2019.420044
2015-01-19    2020.985046
2015-01-20    2022.550049
2015-01-21    2032.119995
2015-01-22    2063.149902
2015-01-23    2051.820068
2015-01-26    2057.090088
2015-01-27    2029.550049
2015-01-28    2002.160034
2015-01-29    2021.250000
2015-01-30    1994.989990
2015-02-02    2020.849976
2015-02-03    2050.030029
2015-02-04    2041.510010
2015-02-05    2062.520020
2015-02-06    2055.469971
2015-02-09    2046.739990
2015-02-10    2068.590088
2015-02-11    2068.530029
2015-02-12    2088.479980
2015-02-13    2096.989990
2015-02-16    2098.665039
2015-02-17    2100.340088
2015-02-18    2099.679932
2015-02-19    2097.449951
2015-02-20    2110.300049
2015-02-23    2109.659912
2015-02-24    2115.479980
2015-02

In [ ]:
asset_sim

array([[27.3325    , 46.759998  , 15.426     ],
       [27.52266245, 47.28317865, 15.43977788],
       [27.58069781, 46.25440253, 15.34445958],
       [27.47126999, 46.91052381, 15.61397121],
       [27.59302863, 47.28059926, 15.86364348],
       [28.02267905, 47.04302202, 15.88894032],
       [28.64653948, 48.01780254, 16.00509444]])

AAPL_Close    27.332500
MSFT_Close    46.759998
AMZN_Close    15.426000
Name: 2015-01-02 00:00:00, dtype: float64

array([[1.]])